In [1]:
import PIL
import cv2
import geotiff
import numpy as np
from NARFU_AI_18.geoparser import SourceImage, SIFT
from NARFU_AI_18.config import CONFIG
import os
from NARFU_AI_18.geoparser import CropImage
from tifffile import imread as tifread
from matplotlib import pyplot as plt

In [17]:
SIFT.setSigma(1)
# SIFT.setNOctaveLayers(3)
# SIFT.setContrastThreshold(0.3)
# SIFT.setNFeatures(1000000)
SIFT.setEdgeThreshold(10.0)
CONFIG.homografy_confidence = 3
CONFIG.layouts_downscale = 6
CONFIG.max_matches_to_calc_proj = 100

In [ ]:
layouts_dir = os.path.join(CONFIG.data_path, CONFIG.layouts_dir_name)
fig = plt.figure(figsize=(10, 7), dpi=400)
index = 1

remove_clouds = np.vectorize(lambda x: x/2000*255 if x < 2000 else 0)

for file_name in os.listdir(layouts_dir):
    fig.add_subplot(2, 2, index)
    index += 1
    image = tifread(os.path.join(layouts_dir, file_name))[:, :, 3]/2000*255
    # image = remove_clouds(image)
    # vals = image.mean(axis=1).flatten()
    # # plot histogram with 255 bins
    # b, bins, patches = plt.hist(vals, 255)
    # plt.xlim([0,255])
    # image = image.clip(0, 1)
    image = np.array(
            PIL.Image.fromarray(np.array(image, dtype=np.uint8)).resize(
                [image.shape[0] // 4, image.shape[1] // 4],
                resample=PIL.Image.BICUBIC,
            ),
            dtype=np.uint8
        )
    cv2.normalize(image, image, 50, 150, cv2.NORM_MINMAX)
    
    plt.imshow(image)
    plt.axis("off")
    plt.title(file_name)
plt.show()


In [ ]:
layouts_dir = os.path.join(CONFIG.data_path, CONFIG.layouts_dir_name)
layouts = [SourceImage(os.path.join(layouts_dir, file_name)) for file_name in os.listdir(layouts_dir)]

In [ ]:
fig = plt.figure(figsize=(10, 7), dpi=400) 
for layout in layouts:
    fig.add_subplot(2, 2, layouts.index(layout)+1) 
    plt.imshow(layout.image)
    plt.axis('off') 
    plt.title(layout.name) 
plt.show()

In [ ]:
crops_path = os.path.join(CONFIG.data_path, CONFIG.crops_dir_name)
for crop_file_name in os.listdir(crops_path):
    crop = tifread(os.path.join(crops_path, crop_file_name))
    crop = np.array(crop[:,:,:3] / 2000 * 255, dtype=np.uint8)
    crop = CropImage(crop, crop_file_name)

    print(crop_file_name)
    for layout in layouts:
        transform, _, matches = crop.match(layout)
        crop_transformed = cv2.warpPerspective(np.copy(crop.image), transform, (layout.image.shape[0], layout.image.shape[1]))
        mask = np.zeros_like(layout.image, dtype = np.uint8)
        roi_corners = np.int32([
        [
            [0, 0],
            [0, layout.image.shape[0] - 1],
            [layout.image.shape[1] - 1, layout.image.shape[0] - 1],
            [layout.image.shape[1] - 1, 0]
        ]
        ])
        ignore_mask_color = (255)
        cv2.fillPoly(mask, roi_corners, ignore_mask_color)
        img2_masked = cv2.bitwise_and(layout.image, mask)
        img_combined = cv2.add(img2_masked, crop_transformed)
        cv2.imshow('image', img_combined)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        coords = crop.get_coords(layout)
        valid = True
        for coord in coords:
            if(coord[2] > 1.2) or (coord[2] < 0.8):
                valid = False

        if(valid):
            print(coords)
        else:
            print('not valid', coords)

In [18]:


crop = tifread('NARFU_AI_18/data/crops/crop_1_1_0000.tif')
crop = crop[:,:,:3] / 3000 * 255
crop = np.array(crop, dtype=np.uint8)
crop = CropImage(crop)
# cv2.imshow('image', crop.image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [19]:
FILE = SourceImage('NARFU_AI_18/data/layouts/layout_2021-08-16.tif')
# cv2.imshow('image', FILE.image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [20]:
M, _, matches = crop.match(FILE)
M

array([[-9.88579039e-01, -4.50247013e+00,  8.68069868e+02],
       [ 2.95262742e-01, -5.53678578e-01,  1.17750697e+02],
       [-8.22700096e-04, -5.44443365e-03,  1.00000000e+00]])

In [12]:
img = FILE.image
sift = cv2.SIFT_create()
keypoints = FILE.keypoints
descr = FILE.descriptors
crop_keypoints = crop.keypoints
crop_descr = crop.descriptors

In [13]:
kimg = cv2.drawKeypoints(img, keypoints, np.copy(img))
cv2.imshow('image', kimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
kimg = cv2.drawKeypoints(crop.image, crop_keypoints, np.copy(crop.image))
cv2.imshow('image', kimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

Qt: Session management error: None of the authentication protocols specified are supported


In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

In [14]:
# matches = bf.match(crop_descr,descr)
# matches = sorted(matches, key = lambda x:x.distance)
matched_img = cv2.drawMatches(crop.image, crop_keypoints, img, keypoints, matches[:50], img, flags=2)
cv2.imshow('image', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
if len(matches) > 50:
   src_pts = np.float32([crop_keypoints[m.queryIdx].pt
      for m in matches[: 50]
   ]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints[m.trainIdx].pt
   for m in matches[: 50]
]).reshape(-1, 1, 2)

In [ ]:
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

In [ ]:
M

In [15]:
crop_transformed = cv2.warpPerspective(crop.image, M, (img.shape[0], img.shape[1]))
cv2.imshow('image', crop_transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
mask = np.zeros_like(img, dtype = np.uint8)
roi_corners = np.int32([
   [
      [0, 0],
      [0, img.shape[0] - 1],
      [img.shape[1] - 1, img.shape[0] - 1],
      [img.shape[1] - 1, 0]
   ]
])
ignore_mask_color = (255, 255,255)
cv2.fillPoly(mask, roi_corners, ignore_mask_color)
img2_masked = cv2.bitwise_and(img, mask)
img_combined = cv2.add(img2_masked, crop_transformed)
cv2.imshow('image', img_combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
left_top_corner = M.dot(np.array([0,0,1]))
right_top_corner = M.dot(np.array([crop.image.shape[0], 0, 1]))
right_bottom_corner = M.dot(np.array([crop.image.shape[0], crop.image.shape[1], 1]))
left_bottom_corner = M.dot(np.array([0, crop.image.shape[1], 1]))

print(left_top_corner, right_top_corner, right_bottom_corner, left_bottom_corner)

In [ ]:
crop.get_coords(FILE)